In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autokeras as ak
import os 
import tensorflow as tf

In [2]:
data = np.load('/work/n1000000_0804_all_flat.npz')

In [3]:
input_state = 2
target = data['delta']
if input_state == 0:
    data_all = data['ve_dune']
elif input_state == 1:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif input_state == 2:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune'], data['ve_t2hk'], data['vu_t2hk'], data['vebar_t2hk'], data['vubar_t2hk']])

x_train = data_all[:10000]
y_train = target[:10000]
x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]
x_test = data_all[900000:]
y_test = target[900000:]

In [4]:
clf = ak.StructuredDataRegressor(overwrite=True, max_trials=50)
clf.fit(x_train, y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=50,
           verbose=1,
           shuffle = True
       )

Trial 50 Complete [00h 01m 17s]
val_loss: 775.70654296875

Best val_loss So Far: 270.8165283203125
Total elapsed time: 00h 55m 17s
INFO:tensorflow:Oracle triggered exit

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/50
157/157 [==============================] - 1s 7ms/step - loss: 20386.1621 - mean_squared_error: 20386.1621
Epoch 2/50
157/157 [==============================] - 1s 8ms/step - loss: 5479.7578 - mean_squared_error: 5479.7578
Epoch 3/50
157/157 [==============================] - 1s 8ms/step - loss: 4336.9062 - mean_squared_error:

In [5]:
model = clf.export_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 244)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 244)               0         
_________________________________________________________________
normalization (Normalization (None, 244)               489       
_________________________________________________________________
dense (Dense)                (None, 32)                7840      
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4224      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128)              

In [ ]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 89s 7ms/step - loss: 195.0002 - mean_squared_error: 195.0002 - val_loss: 157.1698 - val_mean_squared_error: 157.1698
Epoch 2/20
12516/12516 [==============================] - 87s 7ms/step - loss: 168.1634 - mean_squared_error: 168.1634 - val_loss: 110.2250 - val_mean_squared_error: 110.2250
Epoch 3/20
12516/12516 [==============================] - 87s 7ms/step - loss: 154.3117 - mean_squared_error: 154.3117 - val_loss: 94.9089 - val_mean_squared_error: 94.9089
Epoch 4/20
12516/12516 [==============================] - 87s 7ms/step - loss: 150.5822 - mean_squared_error: 150.5822 - val_loss: 190.7868 - val_mean_squared_error: 190.7868
Epoch 5/20
 3563/12516 [=======>......................] - ETA: 57s - loss: 141.8218 - mean_squared_error: 141.8218

In [ ]:
model.evaluate(x_test, y_test)
pre_test = model.predict(x_test)

In [ ]:
plt.hist(pre_test, histtype = 'step', bins = 100, label = 'prediction')
plt.hist(y_test, histtype = 'step', bins = 100, label = 'target')
plt.xlabel('delta')
plt.ylabel('number')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(pre_test, y_test, '.', alpha = 0.05)
plt.xlabel('prediction')
plt.ylabel('target')
plt.title('delta')

In [ ]:
if input_state == 0:
    modelname = '/work/Regression/models_ve_dune/0803_delta4.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 1:
    modelname = '/work/Regression/models_all_dune/0803_delta2.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)
elif input_state == 2:
    modelname = '/work/Regression/models_all/0804_delta2.h5'
    if not os.path.isfile(modelname):
        model.save(modelname)